# Traitement des catégories

Ce notebook présente différentes options pour gérer les catégories au format entier ou texte.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline

On construit un jeu très simple avec deux catégories, une entière, une au format texte.

In [3]:
import pandas
import numpy
df = pandas.DataFrame(dict(cat_int=[10, 20, 10, 39, 10, 10, numpy.nan],
                          cat_text=['catA', 'catB', 'catA', 'catDD', 'catB', numpy.nan, 'catB']))
df

,cat_int,cat_text
0,10.0,catA
1,20.0,catB
2,10.0,catA
3,39.0,catDD
4,10.0,catB
5,10.0,NaN
6,NaN,catB


## Transformations d'une catégorie

Les premières opérations consiste à convertir une catégorie au format entier ou au format texte en un entier. Les valeurs manquantes ne sont toujours traitées de la même façon.

In [4]:
from sklearn.preprocessing import LabelEncoder
LabelEncoder().fit_transform(df['cat_int'])

array([0, 1, 0, 2, 0, 0, 3], dtype=int64)

In [5]:
try:
    LabelEncoder().fit_transform(df['cat_text'])
except Exception as e:
    print(e)
LabelEncoder().fit_transform(df['cat_text'].dropna())

'<' not supported between instances of 'float' and 'str'


array([0, 1, 0, 2, 1, 1], dtype=int64)

La seconde opération permet de transformer une catégorie au format entier en plusieurs colonnes au format binaire.

In [6]:
from sklearn.preprocessing import OneHotEncoder
try:
    OneHotEncoder().fit_transform(df[['cat_int']]).todense()
except Exception as e:
    print(e)
OneHotEncoder().fit_transform(df[['cat_int']].dropna()).todense()

Input contains NaN, infinity or a value too large for dtype('float64').


matrix([[1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [7]:
from sklearn.preprocessing import LabelBinarizer
try:
    LabelBinarizer().fit_transform(df[['cat_int']])
except Exception as e:
    print(e)
LabelBinarizer().fit_transform(df[['cat_int']].dropna())

Unknown label type: (   cat_int
0     10.0
1     20.0
2     10.0
3     39.0
4     10.0
5     10.0
6      NaN,)


array([[1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]])

In [8]:
from sklearn.preprocessing import LabelBinarizer
try:
    LabelBinarizer().fit_transform(df[['cat_text']])
except Exception as e:
    print(e)
LabelBinarizer().fit_transform(df[['cat_text']].dropna())

'<' not supported between instances of 'float' and 'str'


array([[1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0]])

D'autres options qui ne fonctionnent pas tout à fait de la même manière en terme d'implémentation.

In [9]:
from sklearn.feature_extraction import DictVectorizer
DictVectorizer().fit_transform(df.to_dict('records')).todense()

matrix([[10.,  0.,  1.,  0.,  0.],
        [20.,  0.,  0.,  1.,  0.],
        [10.,  0.,  1.,  0.,  0.],
        [39.,  0.,  0.,  0.,  1.],
        [10.,  0.,  0.,  1.,  0.],
        [10., nan,  0.,  0.,  0.],
        [nan,  0.,  0.,  1.,  0.]])

In [10]:
from sklearn.feature_extraction import FeatureHasher
FeatureHasher(n_features=5).fit_transform(df.to_dict('records')).todense()

matrix([[ 0.,  0.,  0.,  1., 10.],
        [ 1.,  0.,  0.,  0., 20.],
        [ 0.,  0.,  0.,  1., 10.],
        [ 0., -1.,  0.,  0., 39.],
        [ 1.,  0.,  0.,  0., 10.],
        [ 0.,  0.,  0.,  0., nan],
        [ 1.,  0.,  0.,  0., nan]])

## Méthodes à gradient et ensemblistes

On construit un simple jeu de données pour une régression linéaire, *Y = X + catégorie* puis on cale une régression linéaire avec $Y = X2 + \epsilon$ où *X2* est une permutation de *X*. Le lien est en quelque sorte brisé.

In [11]:
perm = numpy.random.permutation(list(range(10)))
n = 1000
X1 = numpy.random.randint(0, 10, (n,1))
X2 = numpy.array([perm[i] for i in X1])
eps = numpy.random.random((n, 1))
Y = X1 * (-10) - 7 + eps
data = pandas.DataFrame(dict(X1=X1.ravel(), X2=X2.ravel(), Y=Y.ravel()))
data.head()

,X1,X2,Y
0,2,5,-26.768848
1,3,8,-36.990922
2,5,3,-56.783432
3,5,3,-56.144648
4,8,6,-86.168897


In [12]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data)
data_train = data_train.copy()
data_test = data_test.copy()

On transforme la catégorie :

In [13]:
le = LabelEncoder().fit(data_train['X2'])
data_train['X3'] = le.transform(data_train['X2'])
data_test['X3'] = le.transform(data_test['X2'])
data_train.head()

,X1,X2,Y,X3
199,5,3,-56.841166,3
368,7,4,-76.726482,4
169,3,8,-36.808215,8
219,8,6,-86.812655,6
381,0,1,-6.415813,1


In [14]:
data_train.corr()

,X1,X2,Y,X3
X1,1.000000,0.123821,-0.999950,0.123821
X2,0.123821,1.000000,-0.124095,1.000000
Y,-0.999950,-0.124095,1.000000,-0.124095
X3,0.123821,1.000000,-0.124095,1.000000


On cale une régression linéaire :

In [15]:
from sklearn.linear_model import LinearRegression
clr = LinearRegression()
clr.fit(data_train[['X3']], data_train['Y'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [16]:
from sklearn.metrics import r2_score
r2_score(data_test['Y'], clr.predict(data_test[['X3']]))

-0.01646154619599427

On cale un arbre de décision :

In [17]:
from sklearn.tree import DecisionTreeRegressor
clr = DecisionTreeRegressor()
clr.fit(data_train[['X3']], data_train['Y'])

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [18]:
r2_score(data_test['Y'], clr.predict(data_test[['X3']]))

0.9998913992412252

L'arbre de décision a saisi la permutation alors que la régression linéaire n'a pas fonctionné. La régression linéaire n'est pas estimée à l'aide d'une méthode à base de gradient mais elle possède les mêmes contraintes, il est préférable que la cible *Y* soit une fonction le plus possible monotone de *X*. Avec une colonne par modalité de la catégorie, le résultat est tout autre.

In [19]:
one = OneHotEncoder().fit(data_train[['X2']])
feat = one.transform(data_train[['X2']])
feat[:5].todense()

matrix([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [20]:
clr = LinearRegression()
clr.fit(feat, data_train['Y'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
r2_score(data_test['Y'], clr.predict(one.transform(data_test[['X2']])))

0.9998913992412252